In [ ]:
import pandas as pd
data_NotFound=pd.read_pickle("22patents_NotFound.pkl")

In [3]:
data_NotFound

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
0,1340219.0,302182616.0,1991-08-20,EP,Y,NaN,0
1,1762664.0,303166397.0,1989-03-03,EP,Y,NaN,0
2,3684009.0,290188743.0,2000-06-09,EP,Y,NaN,0
3,4151350.0,302315673.0,1990-10-19,EP,Y,NaN,0
4,4154642.0,301078296.0,1993-03-16,EP,Y,NaN,0
...,...,...,...,...,...,...,...
120,45604812.0,407719213.0,2010-08-17,EP,Y,NaN,0
121,46148699.0,378503932.0,2011-05-30,EP,N,NaN,0
122,46799002.0,413546805.0,2012-07-27,EP,N,NaN,0
123,48225195.0,406892642.0,2010-08-03,EP,N,NaN,0


In [4]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    hosts=f"https://es2.logic-mill.net:443",
    basic_auth=("elastic", "MTqCwEli2DWAJTFGKwIa"),
    retry_on_status=[500, 501, 502, 503, 504, 524],
    retry_on_timeout=True,
    request_timeout=60,
)

In [6]:
# Fetch the embedding for the given patent ID
res = client.get(
    index='patents',
    id=48613386,
)
embedding = res["_source"]["paecter_embedding"]
k = 2000
target_date = res["_source"]["earliest_priority_date"]  #publication date of the focal patent

# Construct the search query with KNN and date filter
query = {
    "size": k,
    "query": {
        "bool": {
            "must": [
                {  # kNN Search
                    "knn": {
                        "field": "paecter_embedding",
                        "num_candidates": min(k * 2, 10_000),  # Increase num_candidates
                        "query_vector": embedding
                    }
                },
                {  # Date Filter
                    "range": {
                        "earliest_filing_date": {
                            "lte": target_date
                        }
                    }
                }
            ]
        }
    },
    "_source": ["earliest_filing_date"]
}


# Perform the search
search = client.search(
    index="patents",
    body=query
)
# Print the search results
result=search["hits"]["hits"]
print(result)
print(len(result))

NotFoundError: NotFoundError(404, "{'_index': 'patents_v2', '_id': '48613386', 'found': False}")

In [ ]:
data_lessthan1000=pd.read_pickle("22patents_lessthan1000.pkl")
data_lessthan1000.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
0,3501661.0,306304160.0,1986-04-03,EP,N,"[(20346245, 1.9824321), (20344779, 1.9822505),...",824
1,3509044.0,315990994.0,1982-03-29,EP,N,"[(3509044, 1.9997115), (6050144, 1.9730129), (...",670
2,3509480.0,315987871.0,1982-03-30,EP,Y,"[(3509480, 1.9996388), (6142139, 1.978461), (9...",826
3,3519118.0,315990571.0,1983-05-09,EP,N,"[(38572048, 1.9815624), (9113092, 1.9810143), ...",988
4,3522302.0,309121018.0,1982-05-11,EP,Y,"[(3522302, 1.9996936), (20336721, 1.9782782), ...",604


In [15]:
# Fetch the embedding for the given patent ID
res = client.get(
    index='patents',
    id=3501661,
)
embedding = res["_source"]["paecter_embedding"]
k = 3000
target_date = res["_source"]["earliest_priority_date"]  #publication date of the focal patent

# Construct the search query with KNN and date filter
query = {
    "size": k,
    "query": {
        "bool": {
            "must": [
                {  # kNN Search
                    "knn": {
                        "field": "paecter_embedding",
                        "num_candidates": min(k * 2, 10_000),  # Increase num_candidates
                        "query_vector": embedding
                    }
                },
                {  # Date Filter
                    "range": {
                        "earliest_filing_date": {
                            "lte": target_date
                        }
                    }
                }
            ]
        }
    },
    "_source": ["earliest_filing_date"]
}


# Perform the search
search = client.search(
    index="patents",
    body=query
)
# Print the search results
result=search["hits"]["hits"]
print(result)
print(len(result))

[{'_index': 'patents_v2', '_id': '20346245', '_score': 1.9824321, '_source': {'earliest_filing_date': '1982-03-12'}}, {'_index': 'patents_v2', '_id': '20344779', '_score': 1.9822505, '_source': {'earliest_filing_date': '1981-10-13'}}, {'_index': 'patents_v2', '_id': '20348617', '_score': 1.9810977, '_source': {'earliest_filing_date': '1982-11-10'}}, {'_index': 'patents_v2', '_id': '6130454', '_score': 1.9807644, '_ignored': ['abstract.keyword'], '_source': {'earliest_filing_date': '1981-04-18'}}, {'_index': 'patents_v2', '_id': '6060976', '_score': 1.9801276, '_source': {'earliest_filing_date': '1979-01-19'}}, {'_index': 'patents_v2', '_id': '26657979', '_score': 1.9798851, '_source': {'earliest_filing_date': '1982-10-12'}}, {'_index': 'patents_v2', '_id': '66346894', '_score': 1.9797482, '_source': {'earliest_filing_date': '1983-05-17'}}, {'_index': 'patents_v2', '_id': '66344717', '_score': 1.9794121, '_source': {'earliest_filing_date': '1982-08-04'}}, {'_index': 'patents_v2', '_id':

In [12]:
data_lessthan1000.drop(['similar_patents', 'num_unique_patents'], axis=1,inplace=True)
data_lessthan1000.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted
0,3501661.0,306304160.0,1986-04-03,EP,N
1,3509044.0,315990994.0,1982-03-29,EP,N
2,3509480.0,315987871.0,1982-03-30,EP,Y
3,3519118.0,315990571.0,1983-05-09,EP,N
4,3522302.0,309121018.0,1982-05-11,EP,Y


In [13]:
import numpy as np
data_lessthan1000['similar_patents']=np.nan
data_lessthan1000['similar_patents']=data_lessthan1000['similar_patents'].astype('object')
ids=list(data_lessthan1000['docdb_family_id'])
data_lessthan1000.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents
0,3501661.0,306304160.0,1986-04-03,EP,N,NaN
1,3509044.0,315990994.0,1982-03-29,EP,N,NaN
2,3509480.0,315987871.0,1982-03-30,EP,Y,NaN
3,3519118.0,315990571.0,1983-05-09,EP,N,NaN
4,3522302.0,309121018.0,1982-05-11,EP,Y,NaN


In [14]:
ids

[3501661.0,
 3509044.0,
 3509480.0,
 3519118.0,
 3522302.0,
 3522321.0,
 3563849.0,
 3606125.0,
 3693129.0,
 3693391.0,
 3700690.0,
 3767822.0,
 3768144.0,
 3768980.0,
 3769355.0,
 3769878.0,
 3770528.0,
 3770642.0,
 3770816.0,
 3771146.0,
 3774279.0,
 3775441.0,
 3826140.0,
 3843217.0,
 3843834.0,
 3861979.0,
 3865435.0,
 3874800.0,
 3883274.0,
 4018733.0,
 4053169.0,
 4114048.0,
 4114590.0,
 4121611.0,
 4122905.0,
 4129866.0,
 4177212.0,
 4181137.0,
 4186109.0,
 4188383.0,
 4198000.0,
 4208824.0,
 4227707.0,
 4230155.0,
 4241866.0,
 4246622.0,
 4248510.0,
 4251678.0,
 4261423.0,
 4263448.0,
 4269413.0,
 4271790.0,
 4278166.0,
 4278580.0,
 4278671.0,
 4280969.0,
 4286032.0,
 4300768.0,
 4305101.0,
 4335944.0,
 4341450.0,
 4361535.0,
 4404462.0,
 5462575.0,
 6025240.0,
 6032305.0,
 6032675.0,
 6034367.0,
 6034775.0,
 6035084.0,
 6035471.0,
 6036616.0,
 6036622.0,
 6039334.0,
 6043503.0,
 6044321.0,
 6048389.0,
 6053266.0,
 6056197.0,
 6056681.0,
 6063206.0,
 6064776.0,
 6067724.0,
 607

In [16]:
len(ids)

1130

In [17]:
error_ids=[]
h=-1
for i in ids:
    try:
        h=h+1
        print(h)
        # Fetch the embedding for the given patent ID
        res = client.get(
            index='patents',
            id=int(i),
        )
        embedding = res["_source"]["paecter_embedding"]
        k = 3000
        target_date = res["_source"]["earliest_priority_date"]  #publication date of the focal patent

        # Construct the search query with KNN and date filter
        query = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {  # kNN Search
                            "knn": {
                                "field": "paecter_embedding",
                                "num_candidates": min(k * 2, 10_000),  # Increase num_candidates
                                "query_vector": embedding
                            }
                        },
                        {  # Date Filter
                            "range": {
                                "earliest_filing_date": {
                                    "lte": target_date
                                }
                            }
                        }
                    ]
                }
            },
            "_source": ["earliest_filing_date"]
        }


        # Perform the search
        search = client.search(
            index="patents",
            body=query
        )
        # Print the search results
        result=search["hits"]["hits"]
    
        l=[]
        for j in search["hits"]["hits"]:
            l.append((j['_id'],j["_score"]))
        data_lessthan1000.at[h,'similar_patents']=l
    except:
        error_ids.append(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
data_lessthan1000.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents
0,3501661.0,306304160.0,1986-04-03,EP,N,"[(20346245, 1.9824321), (20344779, 1.9822505),..."
1,3509044.0,315990994.0,1982-03-29,EP,N,"[(3509044, 1.9997115), (6050144, 1.9730129), (..."
2,3509480.0,315987871.0,1982-03-30,EP,Y,"[(3509480, 1.9996388), (6142139, 1.978461), (9..."
3,3519118.0,315990571.0,1983-05-09,EP,N,"[(38572048, 1.9815624), (9113092, 1.9810143), ..."
4,3522302.0,309121018.0,1982-05-11,EP,Y,"[(3522302, 1.9996936), (20336721, 1.9782782), ..."


In [19]:
def unique_patents(lst):
    # Check if lst is a valid list or iterable
    if isinstance(lst, list):
        return len(lst)
    else:
        # Handle the case where the value is not a list (e.g., NaN or float)
        return 0  # or return np.nan if you want to keep it as NaN for invalid entries

In [20]:
data_lessthan1000["num_unique_patents"]=data_lessthan1000["similar_patents"].apply(unique_patents)

In [21]:
data_lessthan1000.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
0,3501661.0,306304160.0,1986-04-03,EP,N,"[(20346245, 1.9824321), (20344779, 1.9822505),...",1320
1,3509044.0,315990994.0,1982-03-29,EP,N,"[(3509044, 1.9997115), (6050144, 1.9730129), (...",1056
2,3509480.0,315987871.0,1982-03-30,EP,Y,"[(3509480, 1.9996388), (6142139, 1.978461), (9...",1531
3,3519118.0,315990571.0,1983-05-09,EP,N,"[(38572048, 1.9815624), (9113092, 1.9810143), ...",1522
4,3522302.0,309121018.0,1982-05-11,EP,Y,"[(3522302, 1.9996936), (20336721, 1.9782782), ...",1182


In [22]:
data_lessthan1000[data_lessthan1000["num_unique_patents"]<1000]

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
7,3606125.0,311448263.0,1977-11-28,EP,Y,"[(3606125, 1.9995618), (3507688, 1.9801404), (...",892
8,3693129.0,306624454.0,1985-09-26,EP,N,"[(6235115, 1.9765546), (24836109, 1.9764559), ...",827
11,3767822.0,310937518.0,1978-11-06,EP,Y,"[(25211103, 1.97283), (10395409, 1.9719071), (...",655
13,3768980.0,310041382.0,1981-03-03,EP,N,"[(24790877, 1.9726346), (5942638, 1.9707608), ...",738
16,3770528.0,387392402.0,1984-03-06,EP,Y,"[(10521832, 1.968519), (5936907, 1.9683444), (...",922
...,...,...,...,...,...,...,...
1113,27550162.0,336578509.0,1981-03-28,EP,N,"[(12986291, 1.9695761), (12166310, 1.9690578),...",866
1116,27565511.0,316124237.0,1987-09-22,EP,N,"[(20344571, 1.9758208), (13487928, 1.9749246),...",587
1118,30128771.0,286163612.0,2002-08-08,EP,Y,"[(26271067, 1.975359), (27215158, 1.9706924), ...",530
1120,32292257.0,307108754.0,1985-04-09,EP,Y,"[(32292257, 1.99961), (9303017, 1.9767675), (8...",976


In [26]:
data_atleast1000_2=data_lessthan1000[data_lessthan1000["num_unique_patents"]>=1000]

In [ ]:
(data_atleast1000_2.reset_index(drop=True)).to_pickle("23patents_atleast1000_2.pkl")

In [28]:
len(data_lessthan1000)

1130

In [29]:
len(data_atleast1000_2)

824

In [30]:
824+306

1130

In [31]:
data_lessthan1000=data_lessthan1000[data_lessthan1000["num_unique_patents"]<1000]

In [32]:
data_lessthan1000

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
7,3606125.0,311448263.0,1977-11-28,EP,Y,"[(3606125, 1.9995618), (3507688, 1.9801404), (...",892
8,3693129.0,306624454.0,1985-09-26,EP,N,"[(6235115, 1.9765546), (24836109, 1.9764559), ...",827
11,3767822.0,310937518.0,1978-11-06,EP,Y,"[(25211103, 1.97283), (10395409, 1.9719071), (...",655
13,3768980.0,310041382.0,1981-03-03,EP,N,"[(24790877, 1.9726346), (5942638, 1.9707608), ...",738
16,3770528.0,387392402.0,1984-03-06,EP,Y,"[(10521832, 1.968519), (5936907, 1.9683444), (...",922
...,...,...,...,...,...,...,...
1113,27550162.0,336578509.0,1981-03-28,EP,N,"[(12986291, 1.9695761), (12166310, 1.9690578),...",866
1116,27565511.0,316124237.0,1987-09-22,EP,N,"[(20344571, 1.9758208), (13487928, 1.9749246),...",587
1118,30128771.0,286163612.0,2002-08-08,EP,Y,"[(26271067, 1.975359), (27215158, 1.9706924), ...",530
1120,32292257.0,307108754.0,1985-04-09,EP,Y,"[(32292257, 1.99961), (9303017, 1.9767675), (8...",976


In [33]:
data_lessthan1000.drop(['similar_patents', 'num_unique_patents'], axis=1,inplace=True)
data_lessthan1000.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted
7,3606125.0,311448263.0,1977-11-28,EP,Y
8,3693129.0,306624454.0,1985-09-26,EP,N
11,3767822.0,310937518.0,1978-11-06,EP,Y
13,3768980.0,310041382.0,1981-03-03,EP,N
16,3770528.0,387392402.0,1984-03-06,EP,Y


In [39]:
data_lessthan1000.reset_index(drop=True,inplace=True)

In [34]:
import numpy as np
data_lessthan1000['similar_patents']=np.nan
data_lessthan1000['similar_patents']=data_lessthan1000['similar_patents'].astype('object')
ids=list(data_lessthan1000['docdb_family_id'])
data_lessthan1000.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents
7,3606125.0,311448263.0,1977-11-28,EP,Y,NaN
8,3693129.0,306624454.0,1985-09-26,EP,N,NaN
11,3767822.0,310937518.0,1978-11-06,EP,Y,NaN
13,3768980.0,310041382.0,1981-03-03,EP,N,NaN
16,3770528.0,387392402.0,1984-03-06,EP,Y,NaN


In [35]:
ids

[3606125.0,
 3693129.0,
 3767822.0,
 3768980.0,
 3770528.0,
 3770642.0,
 3771146.0,
 3843834.0,
 3861979.0,
 3865435.0,
 4114590.0,
 4248510.0,
 4286032.0,
 4341450.0,
 4361535.0,
 4404462.0,
 6036616.0,
 6039334.0,
 6043503.0,
 6067724.0,
 6072607.0,
 6086740.0,
 6095884.0,
 6098942.0,
 6103959.0,
 6108641.0,
 6110007.0,
 6112282.0,
 6112824.0,
 6117603.0,
 6117665.0,
 6127163.0,
 6153576.0,
 6162897.0,
 6188474.0,
 6190367.0,
 6190667.0,
 6201978.0,
 6212139.0,
 6215302.0,
 6233684.0,
 6235162.0,
 6241950.0,
 6253441.0,
 6262815.0,
 6265671.0,
 6282195.0,
 6287018.0,
 6308642.0,
 6318861.0,
 6335900.0,
 6343676.0,
 6350783.0,
 6420155.0,
 6445676.0,
 6515787.0,
 6685498.0,
 6869824.0,
 7867024.0,
 8128037.0,
 8187726.0,
 8188109.0,
 8189457.0,
 8191519.0,
 8195170.0,
 8195937.0,
 8201709.0,
 8206301.0,
 8206491.0,
 9197480.0,
 9221159.0,
 9226144.0,
 9226837.0,
 9238912.0,
 9246352.0,
 9249415.0,
 9249416.0,
 9262072.0,
 9262918.0,
 9270531.0,
 9274027.0,
 9276683.0,
 9290307.0,
 931

In [36]:
len(ids)

306

In [40]:
data_lessthan1000.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents
0,3606125.0,311448263.0,1977-11-28,EP,Y,NaN
1,3693129.0,306624454.0,1985-09-26,EP,N,NaN
2,3767822.0,310937518.0,1978-11-06,EP,Y,NaN
3,3768980.0,310041382.0,1981-03-03,EP,N,NaN
4,3770528.0,387392402.0,1984-03-06,EP,Y,NaN


In [41]:
error_ids=[]
h=-1
for i in ids:
    try:
        h=h+1
        print(h)
        # Fetch the embedding for the given patent ID
        res = client.get(
            index='patents',
            id=int(i),
        )
        embedding = res["_source"]["paecter_embedding"]
        k = 4000
        target_date = res["_source"]["earliest_priority_date"]  #publication date of the focal patent

        # Construct the search query with KNN and date filter
        query = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {  # kNN Search
                            "knn": {
                                "field": "paecter_embedding",
                                "num_candidates": min(k * 2, 10_000),  # Increase num_candidates
                                "query_vector": embedding
                            }
                        },
                        {  # Date Filter
                            "range": {
                                "earliest_filing_date": {
                                    "lte": target_date
                                }
                            }
                        }
                    ]
                }
            },
            "_source": ["earliest_filing_date"]
        }


        # Perform the search
        search = client.search(
            index="patents",
            body=query
        )
        # Print the search results
        result=search["hits"]["hits"]
    
        l=[]
        for j in search["hits"]["hits"]:
            l.append((j['_id'],j["_score"]))
        data_lessthan1000.at[h,'similar_patents']=l
    except:
        error_ids.append(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [42]:
def unique_patents(lst):
    # Check if lst is a valid list or iterable
    if isinstance(lst, list):
        return len(lst)
    else:
        # Handle the case where the value is not a list (e.g., NaN or float)
        return 0  # or return np.nan if you want to keep it as NaN for invalid entries

In [43]:
data_lessthan1000["num_unique_patents"]=data_lessthan1000["similar_patents"].apply(unique_patents)

In [44]:
data_lessthan1000.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
0,3606125.0,311448263.0,1977-11-28,EP,Y,"[(3606125, 1.9995618), (3507688, 1.9801404), (...",1203
1,3693129.0,306624454.0,1985-09-26,EP,N,"[(6235115, 1.9765546), (24836109, 1.9764559), ...",1170
2,3767822.0,310937518.0,1978-11-06,EP,Y,"[(25211103, 1.97283), (10395409, 1.9719071), (...",894
3,3768980.0,310041382.0,1981-03-03,EP,N,"[(24790877, 1.9726346), (5942638, 1.9707608), ...",1104
4,3770528.0,387392402.0,1984-03-06,EP,Y,"[(10521832, 1.968519), (5936907, 1.9683444), (...",1273


In [45]:
data_lessthan1000[data_lessthan1000["num_unique_patents"]<1000]

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
2,3767822.0,310937518.0,1978-11-06,EP,Y,"[(25211103, 1.97283), (10395409, 1.9719071), (...",894
6,3771146.0,387393426.0,1985-06-13,EP,N,"[(10534877, 1.9741318), (10543334, 1.9697537),...",350
9,3865435.0,311405424.0,1978-10-10,EP,Y,"[(3865435, 1.9995732), (11679876, 1.9664657), ...",498
23,6098942.0,310543469.0,1980-04-01,EP,Y,"[(6098942, 1.9995973), (63437064, 1.9703271), ...",620
28,6112824.0,315983462.0,1980-09-23,EP,Y,"[(6112824, 1.999078), (25788081, 1.9807699), (...",989
...,...,...,...,...,...,...,...
297,27494163.0,299802562.0,1993-09-07,EP,N,"[(23584464, 1.9726574), (26159110, 1.9697318),...",806
298,27524585.0,300129744.0,1993-12-27,EP,N,"[(4155784, 1.9776208), (4155693, 1.9774344), (...",629
299,27542360.0,310019362.0,1981-02-12,EP,Y,"[(5844648, 1.9737649), (6004685, 1.9732728), (...",739
302,27565511.0,316124237.0,1987-09-22,EP,N,"[(20344571, 1.9758208), (13487928, 1.9749246),...",833


In [47]:
data_atleast1000_3=data_lessthan1000[data_lessthan1000["num_unique_patents"]>=1000]

In [ ]:
(data_atleast1000_3.reset_index(drop=True)).to_pickle("23patents_atleast1000_3.pkl")

In [49]:
data_lessthan1000=data_lessthan1000[data_lessthan1000["num_unique_patents"]<1000]

In [50]:
data_lessthan1000.drop(['similar_patents', 'num_unique_patents'], axis=1,inplace=True)
data_lessthan1000.head()

/tmp/ipykernel_203950/1915475779.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_lessthan1000.drop(['similar_patents', 'num_unique_patents'], axis=1,inplace=True)


,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted
2,3767822.0,310937518.0,1978-11-06,EP,Y
6,3771146.0,387393426.0,1985-06-13,EP,N
9,3865435.0,311405424.0,1978-10-10,EP,Y
23,6098942.0,310543469.0,1980-04-01,EP,Y
28,6112824.0,315983462.0,1980-09-23,EP,Y


In [51]:
data_lessthan1000.reset_index(drop=True,inplace=True)

In [52]:
import numpy as np
data_lessthan1000['similar_patents']=np.nan
data_lessthan1000['similar_patents']=data_lessthan1000['similar_patents'].astype('object')
ids=list(data_lessthan1000['docdb_family_id'])
data_lessthan1000.head()

/tmp/ipykernel_203950/1061598533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_lessthan1000['similar_patents']=np.nan
/tmp/ipykernel_203950/1061598533.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_lessthan1000['similar_patents']=data_lessthan1000['similar_patents'].astype('object')


,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents
0,3767822.0,310937518.0,1978-11-06,EP,Y,NaN
1,3771146.0,387393426.0,1985-06-13,EP,N,NaN
2,3865435.0,311405424.0,1978-10-10,EP,Y,NaN
3,6098942.0,310543469.0,1980-04-01,EP,Y,NaN
4,6112824.0,315983462.0,1980-09-23,EP,Y,NaN


In [53]:
ids

[3767822.0,
 3771146.0,
 3865435.0,
 6098942.0,
 6112824.0,
 6117603.0,
 6117665.0,
 6127163.0,
 6162897.0,
 6188474.0,
 6201978.0,
 6233684.0,
 6235162.0,
 6241950.0,
 6253441.0,
 6265671.0,
 6282195.0,
 6318861.0,
 6343676.0,
 6420155.0,
 6685498.0,
 8188109.0,
 8195170.0,
 8201709.0,
 9221159.0,
 9226837.0,
 9249416.0,
 9262918.0,
 9274027.0,
 9290307.0,
 9451721.0,
 9482696.0,
 10505151.0,
 10538399.0,
 11257496.0,
 11289352.0,
 11307720.0,
 12438004.0,
 12640153.0,
 12980253.0,
 13991099.0,
 14071058.0,
 15023570.0,
 15150110.0,
 15458263.0,
 16003351.0,
 16007642.0,
 16438592.0,
 16690726.0,
 17950117.0,
 19830227.0,
 19834361.0,
 19835823.0,
 19842598.0,
 19848390.0,
 20335497.0,
 20335692.0,
 20339441.0,
 20342532.0,
 21284804.0,
 21738244.0,
 21761892.0,
 21917029.0,
 21927252.0,
 21981255.0,
 22168174.0,
 22276851.0,
 22300230.0,
 22386453.0,
 22430056.0,
 22434444.0,
 22466053.0,
 22745957.0,
 22820944.0,
 22835012.0,
 22851418.0,
 22898689.0,
 23532729.0,
 23545209.0,
 2406

In [54]:
len(ids)

111

In [55]:
error_ids=[]
h=-1
for i in ids:
    try:
        h=h+1
        print(h)
        # Fetch the embedding for the given patent ID
        res = client.get(
            index='patents',
            id=int(i),
        )
        embedding = res["_source"]["paecter_embedding"]
        k = 6000
        target_date = res["_source"]["earliest_priority_date"]  #publication date of the focal patent

        # Construct the search query with KNN and date filter
        query = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {  # kNN Search
                            "knn": {
                                "field": "paecter_embedding",
                                "num_candidates": min(k * 2, 10_000),  # Increase num_candidates
                                "query_vector": embedding
                            }
                        },
                        {  # Date Filter
                            "range": {
                                "earliest_filing_date": {
                                    "lte": target_date
                                }
                            }
                        }
                    ]
                }
            },
            "_source": ["earliest_filing_date"]
        }


        # Perform the search
        search = client.search(
            index="patents",
            body=query
        )
        # Print the search results
        result=search["hits"]["hits"]
    
        l=[]
        for j in search["hits"]["hits"]:
            l.append((j['_id'],j["_score"]))
        data_lessthan1000.at[h,'similar_patents']=l
    except:
        error_ids.append(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110


In [56]:
def unique_patents(lst):
    # Check if lst is a valid list or iterable
    if isinstance(lst, list):
        return len(lst)
    else:
        # Handle the case where the value is not a list (e.g., NaN or float)
        return 0  # or return np.nan if you want to keep it as NaN for invalid entries
data_lessthan1000["num_unique_patents"]=data_lessthan1000["similar_patents"].apply(unique_patents)
data_lessthan1000.head()

/tmp/ipykernel_203950/4126813411.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_lessthan1000["num_unique_patents"]=data_lessthan1000["similar_patents"].apply(unique_patents)


,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
0,3767822.0,310937518.0,1978-11-06,EP,Y,"[(25211103, 1.97283), (10395409, 1.9719071), (...",1107
1,3771146.0,387393426.0,1985-06-13,EP,N,"[(10534877, 1.9741318), (10543334, 1.9697537),...",473
2,3865435.0,311405424.0,1978-10-10,EP,Y,"[(3865435, 1.9995732), (11679876, 1.9664657), ...",660
3,6098942.0,310543469.0,1980-04-01,EP,Y,"[(6098942, 1.9995973), (63437064, 1.9703271), ...",849
4,6112824.0,315983462.0,1980-09-23,EP,Y,"[(6112824, 1.999078), (25788081, 1.9807699), (...",1316


In [59]:
len(data_lessthan1000[data_lessthan1000["num_unique_patents"]<1000])

57

In [ ]:
data_atleast1000_4=data_lessthan1000[data_lessthan1000["num_unique_patents"]>=1000]
(data_atleast1000_4.reset_index(drop=True)).to_pickle("23patents_atleast1000_4.pkl")
data_lessthan1000=data_lessthan1000[data_lessthan1000["num_unique_patents"]<1000]
data_lessthan1000.drop(['similar_patents', 'num_unique_patents'], axis=1,inplace=True)
data_lessthan1000.reset_index(drop=True,inplace=True)
import numpy as np
data_lessthan1000['similar_patents']=np.nan
data_lessthan1000['similar_patents']=data_lessthan1000['similar_patents'].astype('object')
ids=list(data_lessthan1000['docdb_family_id'])
len(ids)

57

In [61]:
error_ids=[]
h=-1
for i in ids:
    try:
        h=h+1
        print(h)
        # Fetch the embedding for the given patent ID
        res = client.get(
            index='patents',
            id=int(i),
        )
        embedding = res["_source"]["paecter_embedding"]
        k = 8000
        target_date = res["_source"]["earliest_priority_date"]  #publication date of the focal patent

        # Construct the search query with KNN and date filter
        query = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {  # kNN Search
                            "knn": {
                                "field": "paecter_embedding",
                                "num_candidates": min(k * 2, 10_000),  # Increase num_candidates
                                "query_vector": embedding
                            }
                        },
                        {  # Date Filter
                            "range": {
                                "earliest_filing_date": {
                                    "lte": target_date
                                }
                            }
                        }
                    ]
                }
            },
            "_source": ["earliest_filing_date"]
        }


        # Perform the search
        search = client.search(
            index="patents",
            body=query
        )
        # Print the search results
        result=search["hits"]["hits"]
    
        l=[]
        for j in search["hits"]["hits"]:
            l.append((j['_id'],j["_score"]))
        data_lessthan1000.at[h,'similar_patents']=l
    except:
        error_ids.append(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


In [62]:
def unique_patents(lst):
    # Check if lst is a valid list or iterable
    if isinstance(lst, list):
        return len(lst)
    else:
        # Handle the case where the value is not a list (e.g., NaN or float)
        return 0  # or return np.nan if you want to keep it as NaN for invalid entries
data_lessthan1000["num_unique_patents"]=data_lessthan1000["similar_patents"].apply(unique_patents)
len(data_lessthan1000[data_lessthan1000["num_unique_patents"]<1000])

57

In [63]:
data_lessthan1000

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents,num_unique_patents
0,3771146.0,387393426.0,1985-06-13,EP,N,"[(10534877, 1.9741318), (10543334, 1.9697537),...",474
1,3865435.0,311405424.0,1978-10-10,EP,Y,"[(3865435, 1.9995732), (11679876, 1.9664657), ...",658
2,6098942.0,310543469.0,1980-04-01,EP,Y,"[(6098942, 1.9995973), (63437064, 1.9703271), ...",843
3,6162897.0,309310780.0,1982-05-06,EP,Y,"[(6162897, 1.9996037), (15359780, 1.9775226), ...",548
4,6233684.0,315984324.0,1984-04-14,EP,N,"[(6188936, 1.9851768), (6738015, 1.9827847), (...",832
5,6241950.0,315990006.0,1984-07-30,EP,N,"[(6241950, 1.9996803), (6232671, 1.9743323), (...",724
6,6253441.0,337405146.0,1984-12-21,EP,Y,"[(6253441, 1.9995975), (6218022, 1.985343), (6...",925
7,6265671.0,306957438.0,1985-03-19,EP,Y,"[(6265671, 1.9995928), (23079915, 1.9780712), ...",981
8,6318861.0,306192762.0,1987-01-15,EP,N,"[(4124762, 1.9706855), (4791184, 1.9705238), (...",450
9,6343676.0,316011160.0,1987-12-24,EP,Y,"[(6343676, 1.9997272), (10617546, 1.972491), (...",921


In [64]:
data_lessthan1000.drop(['similar_patents', 'num_unique_patents'], axis=1,inplace=True)
data_lessthan1000.reset_index(drop=True,inplace=True)
import numpy as np
data_lessthan1000['similar_patents']=np.nan
data_lessthan1000['similar_patents']=data_lessthan1000['similar_patents'].astype('object')
ids=list(data_lessthan1000['docdb_family_id'])
len(ids)

57

In [65]:
error_ids=[]
h=-1
for i in ids:
    try:
        h=h+1
        print(h)
        # Fetch the embedding for the given patent ID
        res = client.get(
            index='patents',
            id=int(i),
        )
        embedding = res["_source"]["paecter_embedding"]
        k = 10000
        target_date = res["_source"]["earliest_priority_date"]  #publication date of the focal patent

        # Construct the search query with KNN and date filter
        query = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {  # kNN Search
                            "knn": {
                                "field": "paecter_embedding",
                                "num_candidates": min(k * 2, 10_000),  # Increase num_candidates
                                "query_vector": embedding
                            }
                        },
                        {  # Date Filter
                            "range": {
                                "earliest_filing_date": {
                                    "lte": target_date
                                }
                            }
                        }
                    ]
                }
            },
            "_source": ["earliest_filing_date"]
        }


        # Perform the search
        search = client.search(
            index="patents",
            body=query
        )
        # Print the search results
        result=search["hits"]["hits"]
    
        l=[]
        for j in search["hits"]["hits"]:
            l.append((j['_id'],j["_score"]))
        data_lessthan1000.at[h,'similar_patents']=l
    except:
        error_ids.append(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


In [66]:
def unique_patents(lst):
    # Check if lst is a valid list or iterable
    if isinstance(lst, list):
        return len(lst)
    else:
        # Handle the case where the value is not a list (e.g., NaN or float)
        return 0  # or return np.nan if you want to keep it as NaN for invalid entries
data_lessthan1000["num_unique_patents"]=data_lessthan1000["similar_patents"].apply(unique_patents)
len(data_lessthan1000[data_lessthan1000["num_unique_patents"]<1000])

57

In [ ]:
data_lessthan1000.to_pickle("23patents_lessthan1000.pkl")

In [68]:
len(pd.read_pickle("data_lessthan1000.pkl"))+len(pd.read_pickle("data_atleast1000_2.pkl"))+len(pd.read_pickle("data_atleast1000_3.pkl"))+len(pd.read_pickle("data_atleast1000_4.pkl"))

1130